# Aggregate data on scores

Aggregate the synthetic person-level responses to find the mean scores.

## Set-up

### Packages and file paths

In [1]:
# Import required packages
from dataclasses import dataclass
import numpy as np
import os
import pandas as pd

In [2]:
# File paths
@dataclass(frozen=True)
class Paths:
    '''Stores paths to data and files'''
    survey = '../data/survey_data'
    synthetic_data = 'synthetic_data_raw.csv'
    aggregate_score = 'aggregate_scores.csv'


paths = Paths()

In [3]:
# Import functions defined elsewhere
import sys
sys.path.append('../')
from utilities.response_labels import create_response_label_dict
from create_and_process_data.functions import results_by_school_and_group

### Import raw data

In [4]:
data = pd.read_csv(os.path.join(paths.survey, paths.synthetic_data))
data.head()

,gender,transgender,sexual_orientation,neurodivergent,birth_parent1,birth_parent2,birth_you,birth_you_age,autonomy_pressure,autonomy_express,...,peer_talk_listen_lab,peer_talk_helpful_lab,peer_talk_if_lab,accept_peer_lab,year_group_lab,fsm_lab,sen_lab,ethnicity_lab,english_additional_lab,school_lab
0,4.0,2.0,6.0,3.0,2.0,1.0,2.0,1.0,2.0,5.0,...,Fully,Somewhat helpful,Very uncomfortable,Not at all,Year 10,Non-FSM,Non-SEN,Ethnic minority,No,School E
1,1.0,2.0,1.0,3.0,3.0,2.0,3.0,NaN,4.0,2.0,...,Mostly,Very helpful,Very uncomfortable,Slightly,Year 10,Non-FSM,Non-SEN,Ethnic minority,NaN,School D
2,NaN,3.0,4.0,1.0,1.0,1.0,1.0,1.0,NaN,4.0,...,Mostly,Very helpful,Very comfortable,Not at all,Year 10,Non-FSM,Non-SEN,White British,No,School E
3,2.0,5.0,5.0,2.0,2.0,2.0,1.0,3.0,1.0,2.0,...,Fully,Somewhat helpful,Uncomfortable,Mostly,Year 10,Non-FSM,NaN,White British,No,School G
4,NaN,3.0,4.0,1.0,1.0,3.0,3.0,2.0,5.0,2.0,...,Slightly,Somewhat helpful,Uncomfortable,Not at all,Year 8,Non-FSM,Non-SEN,NaN,Yes,School B


## Find mean score by school and pupil group

Find mean score by school:
* Overall
* By year
* By gender (this is currently gender from survey, not council, as can see from GM data extract)
* By FSM
* By SEN

Hide results when count < 10.

In [5]:
# Make a list of the columns that provide a score
score_col = [col for col in data.columns if col.endswith('_score')]

In [6]:
# Define function for finding means and counts
def aggregate(df):
    '''
    Aggregate the score columns in the provided dataset, finding the mean and 
    count of non-NaN
    Inputs:
    - df - dataframe containing the score columns
    Output:
    - res - dataframe with mean and count for each score
    '''
    res = pd.DataFrame({
        # Find mean for each score column, ignoring NaN
        'mean': df[score_col].mean(),
        # Count non-NaN so we know the number of pupils used in the mea
        'count': df[score_col].count()}).rename_axis('variable').reset_index()
    return(res)

In [7]:
# Create version where every question has mean NaN and count 0, to use when a
# school has no pupils of a particular subgroup (i.e. no-one in certain 
# FSM/SEN/gender/year)
no_pupils = aggregate(data)
no_pupils['mean'] = np.nan
no_pupils['count'] = 0
no_pupils.head()

,variable,mean,count
0,birth_you_age_score,NaN,0
1,autonomy_score,NaN,0
2,life_satisfaction_score,NaN,0
3,optimism_score,NaN,0
4,wellbeing_score,NaN,0


In [8]:
# Aggregate for each of the possible schools and pupils groups
agg = results_by_school_and_group(
    data=data, agg_func=aggregate, no_pupils=no_pupils)

# Hide results when n<10
agg.loc[agg['count'] < 10, ['mean', 'count']] = np.nan

agg.head()

,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab
0,birth_you_age_score,7.850427,117.0,School A,All,All,All,All
1,autonomy_score,18.220779,77.0,School A,All,All,All,All
2,life_satisfaction_score,5.065041,123.0,School A,All,All,All,All
3,optimism_score,11.841463,82.0,School A,All,All,All,All
4,wellbeing_score,21.466667,75.0,School A,All,All,All,All


## Save results

In [9]:
agg.to_csv(os.path.join(paths.survey, paths.aggregate_score), index=False)